# IP WebCam url

In [2]:
cameraUrl = "http://192.168.0.103:8080/shot.jpg";

# Mobile Camera Streaming

In [3]:
import numpy as np;
import requests;
import cherrypy;
import cv2;
import IPython;
import sys;

In [4]:
# class CameraLocal(VideoStreamBase):
class CameraLocal():
    
    def __init__(self, cameraUrl):
        self.url = cameraUrl;
        
    def getFrame(self):
        img_resp = requests.get(self.url);
        return img_resp;
    
    def getImage(self):
        img = self.getFrame();
        img = np.array(bytearray(img.content), dtype=np.uint8);
        
        img = cv2.imdecode(img, cv2.IMREAD_COLOR);
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        return img;

In [5]:
class CameraWebService():
    
    def __init__(self, camera):
        self.camera = camera;
    
    @cherrypy.expose
    def index(self):
        output = self.camera.getFrame();
        return output

In [6]:
def testCaptureImage():
    video = CameraLocal(cameraUrl);
    
    img = video.getImage();
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR);
    
    cv2.imshow('Android Camera', img);
    cv2.waitKey(0);

In [7]:
def launchService(url):
    service = CameraWebService(CameraLocal(url));
    cherrypy.quickstart(service);

In [10]:
def main():
    url = cameraUrl if len(sys.argv) != 2 else sys.argv[1];
    launchService(url);

In [9]:
# testCaptureImage();
main();